In [10]:
# Importing Essential Libraries

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests as rq
import pandas as pd
import billboard
import json
import datetime
import csv

In [11]:
# Connecting to spotify platform

cid ="bd71a8abc67c44c2b84cb87d03da7ddf" 
secret = "99ff1c63dd2e4d38a4260d75021ba905"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [12]:
# Creating access token for connection

grant_type='client_credentials'

body_params = {'grant_type' : grant_type}

# returns authorization token for API requests
# https://stackoverflow.com/questions/27554994/spotify-api-authentication-with-python

auth = rq.post("https://accounts.spotify.com/api/token", data = body_params, auth = (cid, secret)).json()

#print(auth)
token = auth["access_token"]
print(token)


BQAq3dgVK5S-zoy2mXBRLwiDfTqv5TGBouAY1VCnQht9hbDmW6yEyu85LmfFXDdBlqiC9IN7BYBzLU_bf4c


In [16]:
# Creating empty lists where the results are going to be stored

artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,10000,50):
    #track_results = sp.search(q='year:2014', type='track', limit=50,offset=i)
    track_results = sp.search(q='kill this love', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

In [17]:
print("Number of artists: ",len(artist_name))
print("Number of tracks: ",len(track_id))
print("Popularity: ",len(popularity))

Number of artists:  53
Number of tracks:  53
Popularity:  53


In [18]:
df_tracks = pd.DataFrame({'artist_name':artist_name,
                          'track_name':track_name,
                          'track_id':track_id,
                          'popularity':popularity})
print(df_tracks.shape)
df_tracks.head(30)

(53, 4)


,artist_name,track_name,track_id,popularity
0,BLACKPINK,Kill This Love,18PergoIrGmRyeYxnaXJN2,88
1,BLACKPINK,Don't Know What To Do,0zYqFyhiTj419q56lNsjk0,78
2,BLACKPINK,Kick It,6Ks1e4WEUeOGgnTGZ4IMXo,74
3,BLACKPINK,Hope Not,7mW8ar9hy7GSeH4lohyOKs,71
4,BLACKPINK,DDU-DU DDU-DU - Remix,4HGLNErVXyaDEjHvk3cL5B,70
5,Epik High,Kill This Love,3Cu2cZW292bhfGEl7j6plB,39
6,Emma Heesters,Kill This Love - English Version,263gK4BL5GQPBvTB3XFxFV,38
7,Funguypiano,Kill This Love (Piano Solo),33bUdw03wlfP1kMwiY7LqD,0
8,Master Andross,Kill This Love,7JlUwCaxJGZXFlwAApRlC1,19
9,Nightcore God,Kill This Love,3GrcRR8uZM515mR4LyJKh3,14


In [ ]:
# Extracting audio features for every track

rows = []
batchsize = 100
None_counter = 0

for i in range(1,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(type = 'artist', batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

#print(feature_results)

print('number of elements in the track_id list:', len(rows))

In [153]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (9999, 18)


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.0752,https://api.spotify.com/v1/audio-analysis/6Ius...,0.504,239320,0.705,6Ius4TC0L3cN74HT7ENE6e,0.000000,6,0.128,-8.205,0,0.3640,175.483,4,https://api.spotify.com/v1/tracks/6Ius4TC0L3cN...,audio_features,spotify:track:6Ius4TC0L3cN74HT7ENE6e,0.584
1,0.6340,https://api.spotify.com/v1/audio-analysis/1CS7...,0.566,241693,0.664,1CS7Sd1u5tWkstBhpssyjP,0.000000,4,0.116,-5.303,0,0.0464,128.945,4,https://api.spotify.com/v1/tracks/1CS7Sd1u5tWk...,audio_features,spotify:track:1CS7Sd1u5tWkstBhpssyjP,0.437
2,0.4310,https://api.spotify.com/v1/audio-analysis/3Jvr...,0.484,204280,0.731,3JvrhDOgAt6p7K8mDyZwRd,0.000000,1,0.151,-6.694,1,0.0379,101.654,4,https://api.spotify.com/v1/tracks/3JvrhDOgAt6p...,audio_features,spotify:track:3JvrhDOgAt6p7K8mDyZwRd,0.510
3,0.5880,https://api.spotify.com/v1/audio-analysis/5Nm9...,0.418,172724,0.420,5Nm9ERjJZ5oyfXZTECKmRt,0.000064,0,0.110,-6.444,1,0.0414,84.094,4,https://api.spotify.com/v1/tracks/5Nm9ERjJZ5oy...,audio_features,spotify:track:5Nm9ERjJZ5oyfXZTECKmRt,0.184
4,0.0930,https://api.spotify.com/v1/audio-analysis/7xoU...,0.628,197267,0.593,7xoUc6faLbCqZO6fQEYprd,0.000002,8,0.096,-5.036,1,0.0323,125.026,4,https://api.spotify.com/v1/tracks/7xoUc6faLbCq...,audio_features,spotify:track:7xoUc6faLbCqZO6fQEYprd,0.104


In [155]:
df_audio_features = df_audio_features.rename(columns={'id': 'track_id'})
df_audio_features.head()

,acousticness,analysis_url,danceability,duration_ms,energy,track_id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.0752,https://api.spotify.com/v1/audio-analysis/6Ius...,0.504,239320,0.705,6Ius4TC0L3cN74HT7ENE6e,0.000000,6,0.128,-8.205,0,0.3640,175.483,4,https://api.spotify.com/v1/tracks/6Ius4TC0L3cN...,audio_features,spotify:track:6Ius4TC0L3cN74HT7ENE6e,0.584
1,0.6340,https://api.spotify.com/v1/audio-analysis/1CS7...,0.566,241693,0.664,1CS7Sd1u5tWkstBhpssyjP,0.000000,4,0.116,-5.303,0,0.0464,128.945,4,https://api.spotify.com/v1/tracks/1CS7Sd1u5tWk...,audio_features,spotify:track:1CS7Sd1u5tWkstBhpssyjP,0.437
2,0.4310,https://api.spotify.com/v1/audio-analysis/3Jvr...,0.484,204280,0.731,3JvrhDOgAt6p7K8mDyZwRd,0.000000,1,0.151,-6.694,1,0.0379,101.654,4,https://api.spotify.com/v1/tracks/3JvrhDOgAt6p...,audio_features,spotify:track:3JvrhDOgAt6p7K8mDyZwRd,0.510
3,0.5880,https://api.spotify.com/v1/audio-analysis/5Nm9...,0.418,172724,0.420,5Nm9ERjJZ5oyfXZTECKmRt,0.000064,0,0.110,-6.444,1,0.0414,84.094,4,https://api.spotify.com/v1/tracks/5Nm9ERjJZ5oy...,audio_features,spotify:track:5Nm9ERjJZ5oyfXZTECKmRt,0.184
4,0.0930,https://api.spotify.com/v1/audio-analysis/7xoU...,0.628,197267,0.593,7xoUc6faLbCqZO6fQEYprd,0.000002,8,0.096,-5.036,1,0.0323,125.026,4,https://api.spotify.com/v1/tracks/7xoUc6faLbCq...,audio_features,spotify:track:7xoUc6faLbCqZO6fQEYprd,0.104


In [156]:
final_data = pd.merge(df_audio_features, df_tracks, on='track_id', how='outer')

In [157]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10852 entries, 0 to 10851
Data columns (total 21 columns):
acousticness        10851 non-null float64
analysis_url        10851 non-null object
danceability        10851 non-null float64
duration_ms         10851 non-null float64
energy              10851 non-null float64
track_id            10852 non-null object
instrumentalness    10851 non-null float64
key                 10851 non-null float64
liveness            10851 non-null float64
loudness            10851 non-null float64
mode                10851 non-null float64
speechiness         10851 non-null float64
tempo               10851 non-null float64
time_signature      10851 non-null float64
track_href          10851 non-null object
type                10851 non-null object
uri                 10851 non-null object
valence             10851 non-null float64
artist_name         10852 non-null object
track_name          10852 non-null object
popularity          10852 non-null int

In [158]:
final_data.drop_duplicates(subset=['artist_name','track_name'], inplace=True)
print('Shape after dropping: ', final_data.shape)

Shape after dropping:  (9153, 21)


In [159]:
# Creating audio features file

final_data.to_csv('SpotifyAudioFeatures_2014.csv',encoding='UTF')

In [160]:
#chart = billboard.ChartData('billboard-200/2018-02-01')
new_chart = billboard.ChartData('hot-100/2014-03-01')

#print(new_chart)

In [161]:
# Converting new_chart to pandas dataframe

billboard_df = pd.DataFrame()

for item in new_chart:
    billboard_df = billboard_df.append(pd.Series(item, index=['track_name']), ignore_index=True)

In [162]:
billboard_df.head()

,track_name
0,'Dark Horse' by Katy Perry Featuring Juicy J
1,'Happy' by Pharrell Williams
2,'Talk Dirty' by Jason Derulo Featuring 2 Chainz
3,'Say Something' by A Great Big World & Christi...
4,'Drunk In Love' by Beyonce Featuring Jay Z


In [163]:
bb_tracks = list(billboard_df.track_name)

In [164]:
count = 0

def find_target(row):
    lst = bb_tracks
    if row['track_name'] in str(lst):
        return 1
        count +=1 
    else:
        return 0

In [165]:
final_data = pd.read_csv('SpotifyAudioFeatures_2014.csv',index_col=0)

In [166]:
final_data['bbhot'] = final_data.apply(find_target, axis=1)

In [15]:
#final_data.head(50)

In [168]:
final_data.to_csv('Spotify_final_2014.csv')

In [169]:
## Final file processing

aggregate_file = pd.read_csv("Spotify_Final.csv")

In [172]:
aggregate_file.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [173]:
print('Shape after dropping: ', aggregate_file.shape)

Shape after dropping:  (51410, 22)


In [1]:
## Final Merge between Spotify and billboard

In [65]:
billboard_dataframe = pd.DataFrame()

In [66]:
billboard_dataframe = pd.read_csv('billboard_cleaned_final.csv')

In [67]:
billboard_dataframe.shape

(1492, 3)

In [68]:
spotify_dataframe = pd.DataFrame()

In [69]:
spotify_dataframe = pd.read_csv('SpotifyAudioFeatures_2014.csv',encoding='UTF')

In [70]:
bb_tracks = list(billboard_dataframe.track_name)

In [71]:
def find_target(row):
    lst = bb_tracks
    if row['track_name'] in str(lst):
        return 1
        count +=1 
    else:
        return 0

In [72]:
spotify_dataframe['bbhot'] = spotify_dataframe.apply(find_target, axis=1)

In [73]:
spotify_dataframe.to_csv('SpotifyAudioFeatures_2014.csv')

In [86]:
#final_df = pd.concat(map(pd.read_csv, ['SpotifyAudioFeatures_2019.csv', 'SpotifyAudioFeatures_2018.csv','SpotifyAudioFeatures_2017.csv']))

In [139]:
## File Creation Block Begins

spotify_2019 = pd.read_csv('SpotifyAudioFeatures_2019.csv',encoding = 'UTF-8')
spotify_2018 = pd.read_csv('SpotifyAudioFeatures_2018.csv',encoding = 'UTF-8')
spotify_2017 = pd.read_csv('SpotifyAudioFeatures_2017.csv',encoding = 'UTF-8')
spotify_2016 = pd.read_csv('SpotifyAudioFeatures_2016.csv',encoding = 'UTF-8')
spotify_2015 = pd.read_csv('SpotifyAudioFeatures_2015.csv',encoding = 'UTF-8')
spotify_2014 = pd.read_csv('SpotifyAudioFeatures_2014.csv',encoding = 'UTF-8')

In [140]:
#spotify_2018.info()
spotify_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8701 entries, 0 to 8700
Data columns (total 22 columns):
acousticness        8700 non-null float64
analysis_url        8700 non-null object
danceability        8700 non-null float64
duration_ms         8700 non-null float64
energy              8700 non-null float64
track_id            8701 non-null object
instrumentalness    8700 non-null float64
key                 8700 non-null float64
liveness            8700 non-null float64
loudness            8700 non-null float64
mode                8700 non-null float64
speechiness         8700 non-null float64
tempo               8700 non-null float64
time_signature      8700 non-null float64
track_href          8700 non-null object
type                8700 non-null object
uri                 8700 non-null object
valence             8700 non-null float64
artist_name         8701 non-null object
track_name          8701 non-null object
popularity          8701 non-null int64
bbhot               

In [141]:
spotify_2019 = spotify_2019.drop_duplicates(subset=['artist_name','track_name'], inplace=False)
spotify_2018 = spotify_2018.drop_duplicates(subset=['artist_name','track_name'], inplace=False)
spotify_2017 = spotify_2017.drop_duplicates(subset=['artist_name','track_name'], inplace=False)
spotify_2016 = spotify_2016.drop_duplicates(subset=['artist_name','track_name'], inplace=False)
spotify_2015 = spotify_2015.drop_duplicates(subset=['artist_name','track_name'], inplace=False)
spotify_2014 = spotify_2014.drop_duplicates(subset=['artist_name','track_name'], inplace=False)

In [142]:
spotify_2019.shape
spotify_2018.shape
spotify_2017.shape
spotify_2016.shape
spotify_2015.shape
spotify_2014.shape

(9153, 22)

In [143]:
#final_frame = [spotify_2019, spotify_2018, spotify_2017, spotify_2016, spotify_2015]

final_frame = [spotify_2019, spotify_2018, spotify_2017, spotify_2016, spotify_2015, spotify_2014]


In [144]:
final_file_frame = pd.concat(final_frame)

In [145]:
final_file_frame.shape

(42763, 22)

In [146]:
final_file_frame = final_file_frame.drop_duplicates(subset=['artist_name','track_name'], inplace=False)

In [147]:
final_file_frame.shape

(42240, 22)

In [148]:
final_file_frame.to_csv('test.csv',encoding = 'UTF-8')